# Arctic

> Arctic helper scripts and functions.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp arctic

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os

import click
from arcticdb import Arctic, LibraryOptions

In [ ]:
# | exports
# | code-fold: true

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()